# Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_theme(style="darkgrid")
import matplotlib.pyplot as plt
import os
from shutil import copyfile

# Functions

In [32]:
def get_sample(data, input_path, output_path, target='movement', 
               n=1000, random_state=123, replace=False, create_directory=False):    
    if target == 'genre':
        data.dropna(axis=0, subset=[target], inplace=True)
    
    sample = data.groupby(by=target).sample(n=n, random_state=random_state, replace=replace)
    
    if create_directory:
        for i, j in sample.iterrows():
            old_path = os.path.join(input_path, j.path)
            new_path = os.path.join(output_path, 
                                    f"{os.path.basename(sample_path)}-{target}-{n}",
                                    eval(f"j.{target}"), j.title)
            os.makedirs(os.path.dirname(new_path), exist_ok=True)
            copyfile(old_path,new_path)
        
        sample["path"] = sample[[target,"image"]].apply(lambda x: "/".join(x),axis=1)
        
    return sample

def get_cs_class(csv_path, target):
    return pd.read_csv(os.path.join(csv_path,target + "_class.txt"), header=None, delim_whitespace=True)
    
def get_cs_train_val(csv_path, target, class_=None):
    
    if not isinstance(class_, pd.DataFrame):
        class_ = get_cs_class(csv_path, target)
        
    cs_train = pd.read_csv(os.path.join(csv_path, target + "_train.csv"), header=None)
    cs_train["split"] = "train"
    
    cs_val = pd.read_csv(os.path.join(csv_path, target + "_val.csv"), header=None)
    cs_val["split"] = "val"
    
    cs = pd.concat([cs_train, cs_val], ignore_index=True)
    cs.columns = ["path", target + "_id", "cs-split-" + target]
        
    cs[target] = cs[target + "_id"].apply(lambda x: class_.loc[x][1])
    cs["style-from-path"] = cs["path"].apply(lambda x: x.split('/')[0])
    cs["artist-from-path"] = cs["path"].apply(lambda x: x.split('/')[1].split('_')[0])
    cs["title-from-path"] = cs["path"].apply(lambda x: x.split('/')[1].split('_')[1])
    
    return cs

def get_data(csv_path, image_path, rm_duplicate=True):
    
    cs_style = get_cs_train_val(csv_path, 'style')
    cs_genre = get_cs_train_val(csv_path, 'genre')
    cs_artist = get_cs_train_val(csv_path, 'artist')

    # There is one duplicata inside genre_train.csv / genre_test.csv
    # One image labelled with two genres
    if rm_duplicate:
        if not cs_genre[cs_genre["path"].duplicated(keep='first')].empty:
            cs_genre.drop(cs_genre[cs_genre["path"].duplicated(keep='first')].index, 
                          inplace=True)
             
    movement_list = [i for i in os.listdir(image_path) 
                     if i != '.DS_Store']
    
    movement = []
    image = []
    artist = []
    title = []
    path = []
    
    for g in movement_list:
        files = os.listdir(os.path.join(image_path,g))
        movement.extend([g]*len(files))
        image.extend(files)
        artist.extend(list(map(lambda x: x.split('_')[0], files)))
        title.extend(list(map(lambda x: x.split('_')[1], files)))
        path.extend(list(map(lambda x: g + '/' + x, files)))
    
    data = pd.DataFrame({"path": path, "movement": movement, "artist": artist, "title": title, "image": image})
    
    data = data.merge(cs_genre[["path", "genre", "cs-split-genre"]], on="path",  how="outer")
    data = data.merge(cs_style[["path", "cs-split-style"]], on="path",  how="outer")
    data = data.merge(cs_artist[["path", "cs-split-artist"]], on="path",  how="outer")

    return data

# Get Data

In [3]:
csv_path = "../raw_data/wikiart"
image_path= "../raw_data/wikiart/wikiart"
sample_path = "../raw_data/wikiart"

In [4]:
data = get_data(csv_path, image_path)

In [5]:
data.head()

,path,movement,artist,title,image,genre,cs-split-genre,cs-split-style,cs-split-artist
0,Early_Renaissance/filippo-lippi_two-saints.jpg,Early_Renaissance,filippo-lippi,two-saints.jpg,filippo-lippi_two-saints.jpg,religious_painting,val,val,NaN
1,Early_Renaissance/filippo-lippi_madonna-del-ce...,Early_Renaissance,filippo-lippi,madonna-del-ceppo.jpg,filippo-lippi_madonna-del-ceppo.jpg,religious_painting,train,train,NaN
2,Early_Renaissance/piero-della-francesca_annunc...,Early_Renaissance,piero-della-francesca,annunciation-1464(1).jpg,piero-della-francesca_annunciation-1464(1).jpg,religious_painting,train,train,NaN
3,Early_Renaissance/pietro-perugino_christ-in-th...,Early_Renaissance,pietro-perugino,christ-in-the-sarcophagus-1473.jpg,pietro-perugino_christ-in-the-sarcophagus-1473...,religious_painting,train,train,NaN
4,Early_Renaissance/antonello-da-messina_madonna...,Early_Renaissance,antonello-da-messina,madonna-and-child-salting-madonna.jpg,antonello-da-messina_madonna-and-child-salting...,religious_painting,val,val,NaN


In [35]:
sample = get_sample(data, image_path, sample_path, target='genre', n=5, random_state=123, 
                    replace=False, create_directory=True)    


In [36]:
sample.head()

,path,movement,artist,title,image,genre,cs-split-genre,cs-split-style,cs-split-artist
77431,abstract_painting/gerhard-richter_red-blue-yel...,Abstract_Expressionism,gerhard-richter,red-blue-yellow.jpg,gerhard-richter_red-blue-yellow.jpg,abstract_painting,train,train,NaN
75656,abstract_painting/sam-francis_black-smudge.jpg,Abstract_Expressionism,sam-francis,black-smudge.jpg,sam-francis_black-smudge.jpg,abstract_painting,train,train,NaN
49856,abstract_painting/theo-van-doesburg_compositio...,Cubism,theo-van-doesburg,composition-i-still-life-1916.jpg,theo-van-doesburg_composition-i-still-life-191...,abstract_painting,val,val,NaN
20664,abstract_painting/sol-lewitt_arcs-from-four-co...,Minimalism,sol-lewitt,arcs-from-four-corners-1986.jpg,sol-lewitt_arcs-from-four-corners-1986.jpg,abstract_painting,train,val,NaN
20115,abstract_painting/angelo-de-sousa_86-3-15q-198...,Minimalism,angelo-de-sousa,86-3-15q-1986.jpg,angelo-de-sousa_86-3-15q-1986.jpg,abstract_painting,train,train,NaN


# Data Visualization

In [ ]:
col = ['cs-split-artist','cs-split-genre','cs-split-style','path']

fig, ax = plt.subplots(1,1,figsize=(15,10))
sns.barplot(y=data[col].count().index, 
            x=data[col].count().values,
            order=data[col].count().sort_values(ascending=False).index,
            ax=ax);

for i, v in enumerate(data[col].count().sort_values(ascending=False).values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_yticklabels(['total','cs-movement (style)','cs-genre','cs-artist']);
ax.set_title("Number of images per target");
ax.set_xlabel("Count");
ax.set_ylabel("Target");

In [ ]:
col = ['artist','genre','movement']

fig, ax = plt.subplots(1,1,figsize=(15,10))
sns.barplot(y=data[col].nunique().index, 
            x=data[col].nunique().values,
            order=data[col].nunique().sort_values(ascending=False).index,
            ax=ax);

for i, v in enumerate(data[col].nunique().sort_values(ascending=False).values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_title("Number of classes per target (all images)");
ax.set_xlabel("Count");
ax.set_ylabel("Target");

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10));
sns.countplot(data=data, y="movement", 
              order = data['movement'].value_counts().index,
              ax=ax);

for i, v in enumerate(data["movement"].value_counts().values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_title("Number of images per movement (style)");
ax.set_ylabel("Movement (style)");

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10));
sns.countplot(data=data, y="genre", 
              order = data['genre'].value_counts().index,
              ax=ax);

for i, v in enumerate(data["genre"].value_counts().values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_title("Number of images per genre");

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10));
sns.barplot(y=data.groupby("movement").artist.nunique().index, 
            x=data.groupby("movement").artist.nunique().values,
            order=data.groupby("movement").artist.nunique().sort_values(ascending=False).index,
            ax=ax);

for i, v in enumerate(data.groupby("movement").artist.nunique().sort_values(ascending=False).values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_title("Number of artists per movement");
ax.set_xlabel("Count");